In [1]:
import numpy as np
import pandas as pd
import os
import json

cwd = os.getcwd()

results_path = cwd[:-28] + "Results/T5_wiki_aug_ft_test/all_scores_range_0_137_trail_{}.npz"
results_path_sbert = cwd[:-28] + "Results/T5_wiki_aug_ft_test/all_sbert_scores_0_137_trail_{}.npz"

results_intent_path = cwd[:-28] + "Results/T5_wiki_aug_ft_test/all_scores_range_by_intent_0_137_trail_{}.txt"

intents = ["What drives the economy in this state?",
           "What are the available modes of transport in this state?",
           "How is the government structured in this state?",
           "What about this state's arts and culture attracts you the most?",
           "The main reasons why you would like living in this state"]

sample_size = 2

model = "T5 Wiki-Transfer Aug FT"

In [2]:

total_data = None
total_data_sbert = []
total_data_intent = {}

for intent in intents:
    total_data_intent[intent] = []

# get all of the "normal" data
for i in range(sample_size):
    temp = np.load(results_path.format(i))
    sbert_temp = np.load(results_path_sbert.format(i))
    data = temp['scores']
    data_sbert = sbert_temp['scores']
    
    if total_data is None:
        # we need to take the mean of axis-one first because each "user-sample" contains a set of predicted summaries. So we first avg those sets, before average across all the data/trials
        total_data = data.mean(axis=1)
        total_data_sbert = data_sbert.mean(axis=1)
    else:
        total_data = np.concatenate((total_data, data.mean(axis=1)), axis=0)
        total_data_sbert = np.concatenate((total_data_sbert, data_sbert.mean(axis=1)), axis=0)

# get all of the "by-intent" data
for i in range(sample_size):
    res = None
    with open(results_intent_path.format(i)) as f:
        res = json.load(f)

    for intent in intents:
        total_data_intent[intent].extend(np.mean(res[intent], axis=1).tolist())

# now calculate the relavent stats
total_mean = np.mean(total_data, axis=0)
total_stddev = np.std(total_data, axis=0)

total_mean_sbert = np.mean(total_data_sbert, axis=0)
total_stddev_sbert = np.std(total_data_sbert, axis=0)

mean_intent = {}
stddev_intent = {}

for intent in intents:
    mean = np.mean(total_data_intent[intent], axis=0)
    stddev = np.std(total_data_intent[intent], axis=0)
    mean_intent[intent] = mean
    stddev_intent[intent] = stddev

metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for i, (row_m, row_s) in enumerate(zip(total_mean, total_stddev)):
    row_str = model + " & "
    for col_m, col_s in zip(row_m, row_s):
        row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
    
    row_str = row_str[:-3]
    row_str += " \\\\"
    print(metrics[i])
    print(row_str)
print("SBERT Cosime Sim. Score: ")
print(model + " & " + str(np.round(total_mean_sbert, 4)) + " $\\pm$ " + str(np.round(total_stddev_sbert, 4)) + " \\\\")

print("\n\n\n")

Rouge-1
T5 Wiki-Transfer Aug FT & 0.1317 $\pm$ 0.0341 & 0.5344 $\pm$ 0.082 & 0.2034 $\pm$ 0.0401 & 0.1271 $\pm$ 0.0251 \\
Rouge-2
T5 Wiki-Transfer Aug FT & 0.0339 $\pm$ 0.0143 & 0.1513 $\pm$ 0.0761 & 0.0534 $\pm$ 0.0221 & 0.0334 $\pm$ 0.0138 \\
Rouge-L
T5 Wiki-Transfer Aug FT & 0.0918 $\pm$ 0.0219 & 0.3799 $\pm$ 0.0694 & 0.1423 $\pm$ 0.0257 & 0.0889 $\pm$ 0.0161 \\
SBERT Cosime Sim. Score: 
T5 Wiki-Transfer Aug FT & 0.484 $\pm$ 0.0726 \\






In [3]:
metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for intent in intents:
    print("\nResults for intent {}\n".format(intent))
    for i, (row_m, row_s) in enumerate(zip(mean_intent[intent], stddev_intent[intent])):
        row_str = "\t"+model + " & "
        for col_m, col_s in zip(row_m, row_s):
            row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
        
        row_str = row_str[:-3]
        row_str += " \\\\"
        print("\t"+metrics[i])
        print(row_str)


Results for intent What drives the economy in this state?

	Rouge-1
	T5 Wiki-Transfer Aug FT & 0.1299 $\pm$ 0.0294 & 0.5262 $\pm$ 0.0799 & 0.202 $\pm$ 0.0349 & 0.1262 $\pm$ 0.0218 \\
	Rouge-2
	T5 Wiki-Transfer Aug FT & 0.0343 $\pm$ 0.0139 & 0.1484 $\pm$ 0.0659 & 0.0541 $\pm$ 0.0218 & 0.0338 $\pm$ 0.0136 \\
	Rouge-L
	T5 Wiki-Transfer Aug FT & 0.0914 $\pm$ 0.0188 & 0.3792 $\pm$ 0.0686 & 0.1428 $\pm$ 0.0226 & 0.0892 $\pm$ 0.0141 \\

Results for intent What are the available modes of transport in this state?

	Rouge-1
	T5 Wiki-Transfer Aug FT & 0.1355 $\pm$ 0.035 & 0.5148 $\pm$ 0.0685 & 0.2056 $\pm$ 0.041 & 0.1285 $\pm$ 0.0256 \\
	Rouge-2
	T5 Wiki-Transfer Aug FT & 0.03 $\pm$ 0.0096 & 0.1211 $\pm$ 0.0429 & 0.0461 $\pm$ 0.0136 & 0.0288 $\pm$ 0.0085 \\
	Rouge-L
	T5 Wiki-Transfer Aug FT & 0.091 $\pm$ 0.0213 & 0.3524 $\pm$ 0.0561 & 0.1385 $\pm$ 0.0245 & 0.0866 $\pm$ 0.0153 \\

Results for intent How is the government structured in this state?

	Rouge-1
	T5 Wiki-Transfer Aug FT & 0.1301 $\pm$ 